Logistic Regression

After tuning hyper parameters on validation set with 8:2 split, I choose L2 penalty and 1.2 penalty weight.

1 Using top frequency feature dimension reduction strategy
Using all features => 78.378%
Using top 3000 features => 78.258%
Using top 2000 features => 77.685%
Using top frequency features dimension reduction strategy can largely reduce computation complexity with slightly loss of accuracy.

2 Using paraent structure dimension reduction strategy
Using all paraent features => 74.195%
Using all paraent features can hold majority information of dataset, but the loss of information reduce accuracy.

3 Using PCA dimension reduction strategy
I use cross validation to determine reduced dimension.
Finally, I reduce dimension to 3000, whicih has the highest validation accuracy.
Test accuracy of 3000 dimension PCA is 78.228%

I use parameters of logistic regression to show each ingredient importance for each cuisine. I get top 5 important ingredients for each cuisine.

In [ ]:
import time
notebookstart= time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [2]:
train_df = pd.read_json('train.json')
test_df = pd.read_json('test.json')
df_index = train_df['id']
y = train_df.cuisine.copy()
train_df.head()
#print(test_df['id'])

cuisine     id                                        ingredients
0        greek  10259  [romaine lettuce, black olives, grape tomatoes...
1  southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...
2     filipino  20130  [eggs, pepper, salt, mayonaise, cooking oil, g...
3       indian  22213                [water, vegetable oil, wheat, salt]
4       indian  13162  [black pepper, shallots, cornflour, cayenne pe...

In [3]:
#print(train_df['ingredients'].apply(','.join))

In [4]:
for _, x in train_df.iterrows():
    for ingredient in x.ingredients:
        ingredient = ingredient.replace(',', '')
for _, x in test_df.iterrows():
    for ingredient in x.ingredients:
        ingredient = ingredient.replace(',', '')

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features = 3000, tokenizer=lambda x: [i.strip() for i in x.split(',')], lowercase=False)
#vect = CountVectorizer(tokenizer=lambda x: [i.strip() for i in x.split(',')], lowercase=False)
x = vect.fit_transform(train_df['ingredients'].apply(','.join)) 
#word_vector_df = pd.DataFrame(x.todense(),columns=vect.get_feature_names())
x_test = vect.transform(test_df['ingredients'].apply(','.join)) 

In [6]:
print("Vocab Length: ", len(vect.get_feature_names()))
print("All Data Shape: ", train_df.shape)
#word_vector_df.index= df_index
#print("Number of Predictors: ", word_vector_df.shape[0],)
#word_vector_df.head()
#for k in word_vector_df.keys():
#    print(k)

Vocab Length:  3000
All Data Shape:  (39774, 3)


In [7]:
print(x_test[0])
test_df.head()
print(x.shape)

  (0, 120)	1
  (0, 211)	1
  (0, 984)	1
  (0, 1752)	1
  (0, 2166)	1
  (0, 2917)	1
(39774, 3000)


In [8]:
split = int(0.8 * x.shape[0])  # Use 80% of the data for training and 20% for validation.
print(f'Training, validation split index: {split}')

Training, validation split index: 31819


In [9]:
X_train, y_train, X_valid, y_valid = x[:split], y[:split], x[split:], y[split:]

In [10]:
print(y_train.shape)

(31819,)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [12]:
model = LogisticRegression(penalty='l2', C = 1.2)
model.fit(X_train, y_train)

LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
train_preds = model.predict(X_train)
valid_preds = model.predict(X_valid)

print(f'Training classification accuracy: {accuracy_score(y_train, train_preds)}')
print(f'Validation classification accuracy: {accuracy_score(y_valid, valid_preds)}')

Training classification accuracy: 0.8646091957635376
Validation classification accuracy: 0.7830295411690761


In [14]:
from sklearn.externals import joblib
joblib.dump(model, './model/0.model')

['./model/0.model']

In [15]:
model.fit(x, y)

LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
train_preds = model.predict(X_train)
valid_preds = model.predict(X_valid)

print(f'Training classification accuracy: {accuracy_score(y_train, train_preds)}')
print(f'Validation classification accuracy: {accuracy_score(y_valid, valid_preds)}')

Training classification accuracy: 0.8583864986328923
Validation classification accuracy: 0.8593337523570082


In [17]:
preds = model.predict(x)

print(f'Training classification accuracy: {accuracy_score(y, preds)}')

Training classification accuracy: 0.858575954140896


In [18]:
train_preds = model.predict(X_train[0])
print(train_preds)

['greek']


In [19]:
test_preds = model.predict(x_test)

In [20]:
print(train_df.id[0:10])

0    10259
1    25693
2    20130
3    22213
4    13162
5     6602
6    42779
7     3735
8    16903
9    12734
Name: id, dtype: int64


In [21]:
submission = pd.DataFrame()
submission['id'] = test_df.id
submission['cuisine'] = pd.Series(test_preds)

In [22]:
submission.head()

id       cuisine
0  18009       british
1  28583   southern_us
2  41580       italian
3  29752  cajun_creole
4  35687       italian

In [23]:
submission.to_csv('predictions.csv', index=False)

In [24]:
cuisine = []
for c in y:
    if c not in cuisine:
        cuisine.append(c)

In [25]:
print(cuisine)

['greek', 'southern_us', 'filipino', 'indian', 'jamaican', 'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai', 'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese', 'irish', 'korean', 'moroccan', 'russian']


In [26]:
param = model.coef_
feature_name = vect.get_feature_names()

In [27]:
most_ingredient = []
most_ingredient_id = []
for i in range(len(cuisine)):
    most_ingredient.append(feature_name[np.argmax(param[i])])
    most_ingredient_id.append(np.argmax(param[i]))

In [28]:
print(most_ingredient)

['cachaca', 'stilton', 'cajun seasoning', 'mandarin oranges', 'calamansi juice', 'gruyere cheese', 'feta cheese', 'curry powder', 'Irish whiskey', 'italian sausage', 'jamaican jerk season', 'miso paste', 'kimchi', 'corn tortillas', 'green tea', 'beets', 'quickcooking grits', 'spanish paprika', 'Thai red curry paste', 'vietnamese fish sauce']


In [29]:
x_small = x[:,most_ingredient_id]

In [30]:
most_ingredient_hist = np.zeros((len(cuisine), len(cuisine)))
for i in range(x_small.shape[0]):
    for j in range(x_small.shape[1]):
        if x_small[i,j] != 0:
            most_ingredient_hist[cuisine.index(y[i]), j] += 1
#print(most_ingredient_hist)

In [31]:
correlation = np.argmax(most_ingredient_hist, axis = 1)
correlation_dict = {}
for i in range(len(cuisine)):
    correlation_dict[cuisine[i]] = correlation[i]
print(correlation_dict)
        

{'greek': 6, 'southern_us': 16, 'filipino': 4, 'indian': 7, 'jamaican': 7, 'spanish': 17, 'italian': 9, 'mexican': 13, 'chinese': 3, 'british': 7, 'thai': 18, 'vietnamese': 19, 'cajun_creole': 2, 'brazilian': 0, 'french': 5, 'japanese': 11, 'irish': 8, 'korean': 12, 'moroccan': 7, 'russian': 15}


In [32]:
correlation_dict['british'] = 1
correlation_dict['jamaican'] = 10
correlation_dict['moroccan'] = 14
print(correlation_dict)

{'greek': 6, 'southern_us': 16, 'filipino': 4, 'indian': 7, 'jamaican': 10, 'spanish': 17, 'italian': 9, 'mexican': 13, 'chinese': 3, 'british': 1, 'thai': 18, 'vietnamese': 19, 'cajun_creole': 2, 'brazilian': 0, 'french': 5, 'japanese': 11, 'irish': 8, 'korean': 12, 'moroccan': 14, 'russian': 15}


In [87]:
def argmax_k(data, k=5):
    index = []
    data_k = []
    for i in range(k):
        if index == []:
            index.append(i)
            data_k.append(data[i])
        else:
            t = len(index)-1
            data_k.append(data[i])
            index.append(i)
            while t >= 0:
                if data[i] > data_k[t]:
                    data_k[t+1] = data_k[t]
                    index[t+1] = index[t]
                    data_k[t] = data[i]
                    index[t] = i
                    t = t - 1 
                else:
                    break
    for i in range(k,len(data),1):
        t = len(index)-1
        while t >= 0:
            if data[i] > data_k[t]:
                if t == len(index)-1:
                    data_k[t] = data[i]
                    index[t] = i
                else:
                    data_k[t+1] = data_k[t]
                    index[t+1] = index[t]
                    data_k[t] = data[i]
                    index[t] = i
                t = t - 1
            else:
                break
    return index

In [90]:
most_k_ingredient_id = []
for i in range(len(cuisine)):
    most_k_ingredient_id.append(argmax_k(param[i], 5))

In [95]:
top_k_ingredient = {}
for i in range(len(cuisine)):
    top_k_ingredient[cuisine[i]] = []
    for k_id in most_k_ingredient_id[correlation_dict[cuisine[i]]]:
        top_k_ingredient[cuisine[i]].append(feature_name[k_id])
print(top_k_ingredient)

{'greek': ['feta cheese', 'feta cheese crumbles', 'greek seasoning', 'greek yogurt', 'greek style plain yogurt'], 'southern_us': ['quickcooking grits', 'grits', 'black-eyed peas', 'collard greens', 'key lime juice'], 'filipino': ['calamansi juice', 'lumpia skins', 'lumpia wrappers', 'condensed milk', 'glutinous rice flour'], 'indian': ['curry powder', 'masala', 'tandoori spices', 'plain yogurt', 'mild curry powder'], 'jamaican': ['jamaican jerk season', 'jerk seasoning', 'jerk sauce', 'allspice', 'ground allspice'], 'spanish': ['spanish paprika', 'chorizo sausage', 'spanish chorizo', 'manchego cheese', 'sherry vinegar'], 'italian': ['italian sausage', 'polenta', 'pesto', 'arborio rice', 'gnocchi'], 'mexican': ['corn tortillas', 'flour tortillas', 'mexican chocolate', 'masa harina', 'tequila'], 'chinese': ['mandarin oranges', 'chinese five-spice powder', 'szechwan peppercorns', 'egg roll wrappers', 'glutinous rice flour'], 'british': ['stilton', 'stilton cheese', 'beef drippings', 'marm

In [114]:
import csv
with open("top5.csv", "w", newline = '') as f:
    writer = csv.writer(f)
    for k,v in top_k_ingredient.items():
        a = []
        a.append(k)
        for item in v:
            a.append(item)
        #for ex in a:
         #   print(type(ex))
        
        writer.writerow(a)
    f.close()